<span style="font-size:1.5em;"><b>Análisis de bases de datos  
Universidad Nacional de Colombia  
2024-2</b></span>
# <b>Siniestralidad Vial en Bogotá: Análisis de Datos para la Gestión de la Movilidad Segura</b>  

<span style="font-size:0.9em;">Laura Nicole Bermudez Santa - labermudezs@unal.edu.co  
David Sebastián Mendoza Cruz - damendozac@unal.edu.co  
David Alejandro Alquichire Rincón - dalquichire@unal.edu.co  
Laura Nicole Bermudez Santa - labermudezs@unal.edu.co  
Juan David Bernal Vesga - jubernalv@unal.edu.co</span>

----

La seguridad vial es un tema de vital importancia para la planificación urbana y la protección de la vida en las ciudades. En Bogotá, la Secretaría Distrital de Movilidad (SDM) ha recopilado datos detallados sobre siniestralidad vial desde el año 2007, los cuales incluyen información sobre lesionados, accidentes, causas, tipos de vehículos involucrados y actores viales. Estos datos, disponibles en la plataforma Enterprise de la SDM, representan una oportunidad única para analizar y comprender los patrones de accidentalidad en la ciudad, con el fin de contribuir a la toma de decisiones informadas en materia de seguridad vial.

Este proyecto tiene los siguientes objetivos:
1. Analizar y clasificar los accidentes de tránsito registrados en Bogotá en la plataforma de la secretaria de movilidad desde el 2007, enfocándose en la relación entre tipo de vehículo, actores viales involucrados, localidades afectadas, el estado de los incidentes, etc. Con el fin de identificar patrones clave y proporcionar insumos para la toma de decisiones en materia de seguridad vial.  
2. Proporcionar herramientas visuales que guíen la toma de decisiones en materia de seguridad vial en resultado del análisis estadístico realizado.
3. Determinar tendencias y generar pronósticos que permitan anticipar posibles escenarios de siniestralidad en el futuro, mediante una proyección temporal del comportamiento de los accidentes de tránsito, considerando variables como la frecuencia de muertes y accidentes simples.
Buscamos lo que se expone en los objetivos, pero la idea es usar datos abiertos expuestos en la página de la secretaria de movilidad donde tenemos acceso a los siguientes datos de siniestralidad:
Lesionado, accidentes, causa, vehiculo y actor.



----

#### **Manipulación y limpieza de las bases de datos individuales**

La Secretaria de movilidad dispone una base de [datos abiertos](https://datos.movilidadbogota.gov.co/maps/ea243e7de8e846c8bd27e47c08771d66/about) amplia en materia de siniestrialidad en Bogotá la cual ha sido alimentada con datos desde el 2007. En esta sección depuraremos los siguientes data sets:
- __*ACCIDENTE*__: nos brinda información detallada de la cantidad de accidentes que se han registrado en la ciudad de bogotá incluyendo sus coordenadas, la clase de choque, fecha y gravedad. (En este dataset se concentra la mayoria de los datos y es de nuestro interes analizarlo a profundidad).
- __*LESIONADO*__: incluye información de las personas involucrada tales como su condición, genero y edad son los datos mas relevantes.
- __*ACC_VIA*__: caracteristicas relevantes de cada via donde sucedio el accidente, resulta relevante al relacionar el estado de las vías en la ciudad de Bogotá como posible causante de siniestros en ciertos lugares o localidades.
- __*ACC_VEHICULO*__: información general del tipo de vehiculo y el uso de este ya sea publico o particular.
- __*ACC_CAUSA*__: incluye una breve descripción de una posible causa asociada al siniestro.
- __*ACC_ACTOR*__: nos brinda información relevante de la condición del actor causante del accidente.

In [1]:
# Importamos las librerias necesarias en Python

import pandas as pd 
import seaborn as sb 
import numpy as np 
import matplotlib.pyplot as plt 
import requests
%matplotlib inline

<p>Dado que le daremos prioridad al data set que contiene la información general de accidentes <b>ACCIDENTES.CSV</b> comenzaremos limpiando y manipulando la información de <b>ACC_VEHICULO</b>, <b>ACC_CAUSA</b> y <b>ACC_ACTOR</b>.<p>

__Traemos *ACC_VEHICULO* que corresponde a la información del vehiculo implicado en el accidente mediante un llamado de API__

In [2]:
# URL de la API
url = "https://sig.simur.gov.co/arcgis/rest/services/Accidentalidad/AccidentalidadAnalisis/FeatureServer/5/query?where=1%3D1&outFields=FORMULARIO,CLASE,SERVICIO&outSR=4326&f=json"

# Hacer la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Convertir la respuesta a JSON
    data = response.json()
    print("Datos obtenidos exitosamente")

Datos obtenidos exitosamente


In [3]:
features = data.get('features', [])  # Obtener la lista de features
rows = []  # Lista para almacenar los datos

# Recorrer cada feature y extraer los atributos
for feature in features:
    attributes = feature.get('attributes', {})  # Obtener el diccionario de atributos
    rows.append(attributes)  # Agregar los atributos a la lista

# Convertir la lista de diccionarios a DataFrame
vehiculo_data = pd.DataFrame(rows)
vehiculo_data.head(5)

,FORMULARIO,CLASE,SERVICIO
0,A000035901,AUTOMOVIL,PARTICULAR
1,A000035901,MICROBUS,PUBLICO
2,A000035531,AUTOMOVIL,PARTICULAR
3,A000035139,CAMIONETA,PUBLICO
4,A000035139,AUTOMOVIL,PARTICULAR


In [4]:
# Verificamos si hay valores nulos
vehiculo_data.isna().sum()

FORMULARIO     0
CLASE         27
SERVICIO      85
dtype: int64

In [ ]:
# Reemplazamos cadenas vacías o espacios con NaN
vehiculo_data.replace(["", "SIN INFORMACION"," ", "NaN", "null"], np.nan, inplace=True)

# Ahora aplicar dropna()
vehiculo_data.dropna(inplace=True)

# Verficamos que se hayan eliminado correctamente
vehiculo_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1915 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   FORMULARIO  1915 non-null   object
 1   CLASE       1915 non-null   object
 2   SERVICIO    1915 non-null   object
dtypes: object(3)
memory usage: 59.8+ KB


El valor en formulario corresponde a un identificador de la información suministrada y a continuación mostramos la variedad de clases de vehiculos que se ven involucrados en accidentes al igual que la clasificación de estos según sus servicios.

In [8]:
print(vehiculo_data['CLASE'].value_counts())
print("")
print(vehiculo_data['SERVICIO'].value_counts())

CLASE
AUTOMOVIL         789
MOTOCICLETA       314
CAMIONETA         248
BUS               208
CAMION, FURGON     98
CAMPERO            83
MICROBUS           77
TRACTOCAMION       38
BUSETA             37
VOLQUETA           23
Name: count, dtype: int64

SERVICIO
PARTICULAR     1022
PUBLICO         847
OFICIAL          45
DIPLOMATICO       1
Name: count, dtype: int64
